# Enyzme Kinetics Example - Michaelis-Menten Kinetics of an Esterase

To run the code, press **shift-Enter** in each box/cell.

### Reaction Equation for Esterase Degrading para-Nitrophenylacetate (pNPA)
The reaction catalyzed by esterase can be written as:

$$
\text{pNPA} \xrightarrow{k_{cat},\ K_m} \text{para-Nitrophenol (pNP)} + \text{Acetate}
$$

Where:
- $  k_{cat} $ is the catalytic rate constant (turnover number),
- $  K_m  $ is the Michaelis constant.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [2]:
# Constants
epsilon = 1000  # Molar absorptivity (M^-1 cm^-1)
path_length = 1  # Path length (cm)
V_max = 100  # Maximum reaction velocity (µM/s)
K_m = 50  # Michaelis constant (µM)
noise_level = 0.02  # Noise level for absorbance data

# Generate time points
time = np.linspace(0, 10, 100)  # 0 to 10 seconds, 100 points

# Substrate concentrations (µM)
substrate_concentrations = np.array([10, 20, 50, 100, 200])

### Michaelis-Menten Equation
The Michaelis-Menten equation describes the reaction velocity $ v $ as a function of substrate concentration $ [S] $:

$$
v = \frac{d[P]}{dt} = \frac{V_{max} [S]}{K_m + [S]}
$$

Where:
- $ v $ is the reaction velocity,
- $ V_{max} $ is the maximum reaction velocity ($ V_{max} = k_{cat} \cdot [E]_0 $),
- $ [S] $ is the substrate concentration,
- $ K_m $ is the Michaelis constant.



In [13]:
# Michaelis-Menten equation
def michaelis_menten(S, V_max, K_m):
    return (V_max * S) / (K_m + S)

### Beer-Lambert Law
The Beer-Lambert law relates absorbance $ A $ to the concentration of the product $ [P] $:

$$
A = \epsilon \cdot [P] \cdot l
$$

Where:
- $ \epsilon $ is the molar absorptivity ( $ M^{-1} cm $ $ ^{-1} $),
- $ [P] $ is the concentration of the product (para-Nitrophenol in this case),
- $ l $ is the path length of the cuvette (cm).


In [ ]:
# Simulate absorbance data
def simulate_absorbance(S, time, epsilon, path_length, noise_level):
    P = michaelis_menten(S, V_max, K_m) * time  # Product concentration
    A = epsilon * path_length * P  # Absorbance
    A_noisy = A + np.random.normal(0, noise_level, size=A.shape)  # Add noise
    return A_noisy

In [ ]:
# Generate data for each substrate concentration
data = []
for S in substrate_concentrations:
    A_noisy = simulate_absorbance(S, time, epsilon, path_length, noise_level)
    data.append(pd.DataFrame({
        'Time': time,
        'Absorbance': A_noisy,
        'Substrate_Concentration': S
    }))

# Combine all data into a single DataFrame
df = pd.concat(data, ignore_index=True)
df

### Concentration from Absorbance
The concentration of the product $ [P] $ can be calculated from the absorbance $ A $:

$$
[P] = \frac{A}{\epsilon \cdot l}
$$


In [ ]:
# Convert absorbance to concentration
df['Concentration'] = df['Absorbance'] / (epsilon * path_length)
df

### Initial Velocity Calculation
The initial velocity $ v_0 $ is calculated as the slope of the concentration vs. time plot at $ t = 0 $:

$$
v_0 = \frac{d[P]}{dt} \bigg|_{t=0}
$$


In [ ]:
# Calculate initial velocities (slope of concentration vs time)
initial_velocities = []
for S in substrate_concentrations:
    subset = df[df['Substrate_Concentration'] == S]
    v0 = np.polyfit(subset['Time'], subset['Concentration'], 1)[0]
    initial_velocities.append(v0)

initial_velocities


### Michaelis-Menten Fit
The Michaelis-Menten equation is fitted to the initial velocities $ v_0 $ at different substrate concentrations $ [S] $:

$$
v_0 = \frac{V_{max} [S]}{K_m + [S]}
$$

The fitted parameters $ V_{max} $ and $ K_m $ are obtained using nonlinear regression.

In [7]:
# Fit Michaelis-Menten equation to initial velocities
popt, _ = curve_fit(michaelis_menten, substrate_concentrations, initial_velocities, p0=[V_max, K_m])


In [ ]:

# Plot the Michaelis-Menten fit
# Define scaling parameter and resolution
scaling_factor = 0.8
dpi = 300

# Plot the Michaelis-Menten fit
plt.figure(figsize=(10 * scaling_factor, 6 * scaling_factor), dpi=dpi)
plt.scatter(substrate_concentrations, initial_velocities, label='Simulated Data')
plt.plot(substrate_concentrations, michaelis_menten(substrate_concentrations, *popt), 
         label=f'Fit: V_max={popt[0]:.2f}, K_m={popt[1]:.2f}', color='red')
plt.xlabel('Substrate Concentration [S] (µM)')
plt.ylabel('Initial Velocity v0 (µM/s)')
plt.title('Michaelis-Menten Fit')
plt.legend()
plt.show()

# Display the DataFrame
print(df.head())

In [ ]:
print(f'Fit: V_max={popt[0]:.2f}, K_m={popt[1]:.2f}')